In [19]:
from lasagne_project_utils import load_data, shared_dataset
import theano
from __future__ import print_function
import sys
import os
import time
import numpy as np
import theano
import theano.tensor as T
import lasagne
import scipy
import timeit
import inspect


In [2]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert inputs.eval().shape[0] == targets.eval().shape[0]
    if shuffle:
        indices = np.arange(inputs.eval().shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, inputs.eval().shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield excerpt
        
        #yield inputs[excerpt], targets[excerpt]

In [3]:
datasets = load_data()

True
/home/ubuntu/data/cifar-10-matlab.tar.gz


In [4]:
batch_size = 200

rng = np.random.RandomState(23455)

X_train, y_train = datasets[0]
X_val, y_val = datasets[1]
X_test, y_test = datasets[2]
#X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()

n_train_batches = X_train.get_value(borrow=True).shape[0]
n_valid_batches = X_val.get_value(borrow=True).shape[0]
n_test_batches = X_test.get_value(borrow=True).shape[0]
n_train_batches //= batch_size
n_valid_batches //= batch_size
n_test_batches //= batch_size

index = T.lscalar()  # index to a [mini]batch

x = T.tensor4('x')
y = T.ivector('y')

channel = 3
imsize= 32

In [21]:
num_epochs = 12

X_train = X_train.reshape((9000,3,32,32))
X_test = X_test.reshape((10000, 3, 32, 32))
X_val = X_val.reshape((1000, 3, 32, 32))

In [6]:
network = lasagne.layers.InputLayer(shape=(None, 3, imsize, imsize), stride=(1,1), pad=1,input_var=x)
print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.Conv2DLayer(
            network, num_filters=96, filter_size=(3, 3),
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(),
            pad=1,
            stride=(1,1))

print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.Conv2DLayer(
            network, num_filters=96, filter_size=(3, 3),
            nonlinearity=lasagne.nonlinearities.rectify,
            pad=1,
            stride=(1,1))

print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.Conv2DLayer(
            network, num_filters=96, filter_size=(3, 3),
            nonlinearity=lasagne.nonlinearities.rectify,
            pad=1, 
            stride=(2,2))

print(lasagne.layers.get_output_shape(network))


network = lasagne.layers.Conv2DLayer(
            network, num_filters=192, filter_size=(3, 3),
            nonlinearity=lasagne.nonlinearities.rectify,
            pad=1,
            stride=(1,1))
print(lasagne.layers.get_output_shape(network))


network = lasagne.layers.Conv2DLayer(
            network, num_filters=192, filter_size=(3, 3),
            nonlinearity=lasagne.nonlinearities.rectify,
            pad=1,
            stride=(1,1))
print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.Conv2DLayer(
            network, num_filters=192, filter_size=(3, 3),
            nonlinearity=lasagne.nonlinearities.rectify,
            pad=1, 
            stride=(2,2))
print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.Conv2DLayer(
            network, num_filters=192, filter_size=(3, 3),
            nonlinearity=lasagne.nonlinearities.rectify,
            stride=(1,1))

print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.Conv2DLayer(
            network, num_filters=192, filter_size=(1, 1),
            nonlinearity=lasagne.nonlinearities.rectify,
            stride=(1,1))
print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.Conv2DLayer(
            network, num_filters=10, filter_size=(1, 1),
            nonlinearity=lasagne.nonlinearities.rectify,
            stride=(1,1))
print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.Pool2DLayer(network, pool_size=(6, 6), mode='average_inc_pad')
print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(network, p=.5),
            num_units=10,
            nonlinearity=lasagne.nonlinearities.softmax)
print(lasagne.layers.get_output_shape(network))


(None, 3, 32, 32)
(None, 96, 32, 32)
(None, 96, 32, 32)
(None, 96, 16, 16)
(None, 192, 16, 16)
(None, 192, 16, 16)
(None, 192, 8, 8)
(None, 192, 6, 6)
(None, 192, 6, 6)
(None, 10, 6, 6)
(None, 10, 1, 1)
(None, 10)


In [10]:
train_prediction = lasagne.layers.get_output(network)
train_loss = lasagne.objectives.categorical_crossentropy(train_prediction, y)
train_loss = train_loss.mean()

params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        train_loss, params, learning_rate=0.01, momentum=0.9)

val_prediction = lasagne.layers.get_output(network)
val_loss = lasagne.objectives.categorical_crossentropy(val_prediction, y)
val_loss = val_loss.mean()
# Create a loss expression for validation/testing. The crucial difference
# here is that we do a deterministic forward pass through the network,
# disabling dropout layers.
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction, y)

test_loss = test_loss.mean()
# As a bonus, also create an expression for the classification accuracy:
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), y),
                  dtype=theano.config.floatX)

# Compile a function performing a training step on a mini-batch (by giving
# the updates dictionary) and returning the corresponding training loss:
train_fn = theano.function([index], 
            train_loss, 
            updates=updates,
            givens={
                x: X_train[index * batch_size: (index + 1) * batch_size],
                y: y_train[index * batch_size: (index + 1) * batch_size]
            }
                           
            
        )

val_fn = theano.function(
        [index],
        val_loss,
        givens={
            x: X_val[index * batch_size: (index + 1) * batch_size],
            y: y_val[index * batch_size: (index + 1) * batch_size]
        }
    )

test_fn = theano.function(
        [index],
        [test_loss, test_acc],
        givens={
            x: X_test[index * batch_size: (index + 1) * batch_size],
            y: y_test[index * batch_size: (index + 1) * batch_size]
        }
    )


# Compile a second function computing the validation loss and accuracy:



In [ ]:
train_nn(train_fn, val_fn, test_fn,
            n_train_batches, n_valid_batches, n_test_batches, num_epochs,
            verbose = True)

training @ iter =  0


In [17]:
def train_nn(train_model, validate_model, test_model,
            n_train_batches, n_valid_batches, n_test_batches, n_epochs,
            verbose = True):
    """
    Wrapper function for training and test THEANO model

    :type train_model: Theano.function
    :param train_model:

    :type validate_model: Theano.function
    :param validate_model:

    :type test_model: Theano.function
    :param test_model:

    :type n_train_batches: int
    :param n_train_batches: number of training batches

    :type n_valid_batches: int
    :param n_valid_batches: number of validation batches

    :type n_test_batches: int
    :param n_test_batches: number of testing batches

    :type n_epochs: int
    :param n_epochs: maximal number of epochs to run the optimizer

    :type verbose: boolean
    :param verbose: to print out epoch summary or not to

    """

    # early-stopping parameters
    patience = 10000  # look as this many examples regardless
    patience_increase = 2  # wait this much longer when a new best is
                           # found
    improvement_threshold = 0.85  # a relative improvement of this much is
                                   # considered significant
    validation_frequency = min(n_train_batches, patience // 2)
                                  # go through this many
                                  # minibatche before checking the network
                                  # on the validation set; in this case we
                                  # check every epoch

    best_validation_loss = np.inf
    best_iter = 0
    test_score = 0.
    start_time = timeit.default_timer()

    epoch = 0
    done_looping = False
    
    while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
        for minibatch_index in range(n_train_batches):

            iter = (epoch - 1) * n_train_batches + minibatch_index

            if (iter % 100 == 0) and verbose:
                print('training @ iter = ', iter)
            cost_ij = train_model(minibatch_index)

            if (iter + 1) % validation_frequency == 0:

                # compute zero-one loss on validation set
                validation_losses = [validate_model(i) for i
                                     in range(n_valid_batches)]
                this_validation_loss = np.mean(validation_losses)

                if verbose:
                    print('epoch %i, minibatch %i/%i, validation error %f %%' %
                        (epoch,
                         minibatch_index + 1,
                         n_train_batches,
                         this_validation_loss/n_valid_batches * 100.))

                # if we got the best validation score until now
                
                if this_validation_loss < best_validation_loss:

                    #improve patience if loss improvement is good enough
                    if this_validation_loss < best_validation_loss *  \
                       improvement_threshold:
                        patience = max(patience, iter * patience_increase)

                    # save best validation score and iteration number
                    best_validation_loss = this_validation_loss
                    best_iter = iter

                    # test it on the test set
                    test_losses = [
                        test_model(i)
                        for i in range(n_test_batches)
                    ]
                    test_score = np.mean(test_losses)

                    if verbose:
                        print(('     epoch %i, minibatch %i/%i, test error of '
                               'best model %f %%') %
                              (epoch, minibatch_index + 1,
                               n_train_batches,
                               test_score/n_test_batches * 100.))

            if patience <= iter:
                done_looping = True
                break

    end_time = timeit.default_timer()

    # Retrieve the name of function who invokes train_nn() (caller's name)
    curframe = inspect.currentframe()
    calframe = inspect.getouterframes(curframe, 2)

    # Print out summary
    print('Optimization complete.')
    print('Best validation score of %f %% obtained at iteration %i, '
          'with test performance %f %%' %
          (best_validation_loss * 100., best_iter + 1, test_score * 100.))
    print(('The training process for function ' +
           calframe[1][3] +
           ' ran for %.2fm' % ((end_time - start_time) / 60.)), file=sys.stderr)


In [20]:

# We could add some weight decay as well here, see lasagne.regularization.

# Create update expressions for training, i.e., how to modify the
# parameters at each training step. Here, we'll use Stochastic Gradient
# Descent (SGD) with Nesterov momentum, but Lasagne offers plenty more.

# Finally, launch the training loop.
num_epochs=10
print("Starting training...")
# We iterate over epochs:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, 500, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, 500, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

# After training, we compute and print the test error:
test_err = 0
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500, shuffle=False):
    inputs, targets = batch
    err, acc = val_fn(inputs, targets)
    test_err += err
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

Starting training...


ValueError: too many values to unpack